# Urban Traffic Bonn
Simulate the urban traffic from the city of Bonn using [ArcGIS API for Python](https://developers.arcgis.com/python/).

In [1]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

## Required Python modules
You need to install ArcGIS API for Python, please follow the [guide](https://developers.arcgis.com/python/guide/anaconda/).

In [12]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from datetime import date, datetime, timedelta
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from geourban.services import aggregate, query, simulations, top
from geourban.types import GridType, VehicleType

## Connect to ArcGIS Online

In [3]:
gis = GIS()

## Create a map showing the city of Bonn

In [4]:
bonn_map = gis.map('Bonn, Germany')
bonn_map

MapView(layout=Layout(height='400px', width='100%'))

# Connect to geourban services
The host parameter must target the specific host like "geoprotests.p.rapidapi.com". Furthermore, the factory directly access `os.environ['x_rapidapi_key']` and uses the specified API key as a header parameter. Otherwise, `georapid.factory.EnvironmentClientFactory.create_client_with_host()` will raise a `ValueError`.

Please, check out the [RapidAPI Account Creation and Management Guide](https://docs.rapidapi.com/docs/account-creation-and-settings).

In [5]:
client = EnvironmentClientFactory.create_client_with_host('geourban.p.rapidapi.com')

# List the available simulations

In [6]:
urban_simulations = simulations(client)
urban_simulations

[{'region': 'DEA2D', 'name': 'Aachen, Stadt', 'date': '2023-12-10'},
 {'region': 'DE246', 'name': 'Bayreuth, Landkreis', 'date': '2023-12-28'},
 {'region': 'DE300', 'name': 'Berlin', 'date': '2023-08-25'},
 {'region': 'DEA22', 'name': 'Bonn, Kreisfreie Stadt', 'date': '2023-08-24'},
 {'region': 'DEE01', 'name': 'Dessau-Roßlau, Stadt', 'date': '2023-08-24'},
 {'region': 'DED21', 'name': 'Dresden, Stadt', 'date': '2023-12-10'},
 {'region': 'DEA12',
  'name': 'Duisburg, Kreisfreie Stadt',
  'date': '2023-12-22'},
 {'region': 'DE113', 'name': 'Esslingen, Landkreis', 'date': '2023-12-19'},
 {'region': 'DE712',
  'name': 'Frankfurt am Main, Kreisfreie Stadt',
  'date': '2023-11-21'},
 {'region': 'DE929', 'name': 'Hannover, Region', 'date': '2023-11-09'},
 {'region': 'DE115', 'name': 'Ludwigsburg, Landkreis', 'date': '2023-12-18'},
 {'region': 'DEA1C', 'name': 'Mettmann, Kreis', 'date': '2023-12-27'},
 {'region': 'DE212',
  'name': 'München, Landeshauptstadt, Kreisfreie Stadt',
  'date': '202

## Request the top 5 accumulated car traffic grid cells
We request these hotspots for the city of Bonn by using the urban region code `DEA22`, the simulation date `2023-08-24`, the vehicle type `Car`, and the grid type `agent`. The returned GeoJSON features represents the grid cells with the highest car throughput.

In [7]:
bonn_region_code = 'DEA22'
simulation_date = date(2023, 8, 24)
vehicle_type = VehicleType.CAR
grid_type = GridType.AGENT
limit = 5
top_traffic_grid_cells = top(client, bonn_region_code, simulation_date, vehicle_type, grid_type, limit=limit)
top_traffic_grid_cells

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[7.07375, 50.74601],
      [7.07504, 50.74601],
      [7.07569, 50.74672],
      [7.07504, 50.74743],
      [7.07375, 50.74743],
      [7.0731, 50.74672],
      [7.07375, 50.74601]]]},
   'properties': {'start_time': '2023-08-24T08:00:00',
    'end_time': '2023-08-24T08:59:59',
    'agent_count': 439}},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[7.07375, 50.74601],
      [7.07504, 50.74601],
      [7.07569, 50.74672],
      [7.07504, 50.74743],
      [7.07375, 50.74743],
      [7.0731, 50.74672],
      [7.07375, 50.74601]]]},
   'properties': {'start_time': '2023-08-24T09:00:00',
    'end_time': '2023-08-24T09:59:59',
    'agent_count': 427}},
  {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[7.07375, 50.74601],
      [7.07504, 50.74601],
      [7.07569, 50.74672],
      [7.07504, 50.74743],
      [7.073

## Convert the returned GeoJSON result into a FeatureSet
The FeatureSet offers direct access to a spatially enabled dataframe. We can easily inspect the time frames (`start_time` - `end_time`) and the number of car vehicles `agent_count`.

In [8]:
bonn_top_traffic_fset = FeatureSet.from_geojson(top_traffic_grid_cells)
top_traffic_sdf = bonn_top_traffic_fset.sdf
top_traffic_sdf

,start_time,end_time,agent_count,OBJECTID,SHAPE
0,2023-08-24T08:00:00,2023-08-24T08:59:59,439,1,"{""rings"": [[[7.07375, 50.74601], [7.07504, 50...."
1,2023-08-24T09:00:00,2023-08-24T09:59:59,427,2,"{""rings"": [[[7.07375, 50.74601], [7.07504, 50...."
2,2023-08-24T07:00:00,2023-08-24T07:59:59,422,3,"{""rings"": [[[7.07375, 50.74601], [7.07504, 50...."
3,2023-08-24T08:00:00,2023-08-24T08:59:59,421,4,"{""rings"": [[[7.07569, 50.74672], [7.07699, 50...."
4,2023-08-24T09:00:00,2023-08-24T09:59:59,408,5,"{""rings"": [[[7.07569, 50.74672], [7.07699, 50...."


## Map the traffic grid cells

In [9]:
top_traffic_sdf.spatial.plot(bonn_map)
bonn_map

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True)

# Query the simulated agents nearby
We are using the center of this crossroad intersection, request the simulated agents being within a distance of `1 kilometer`, and specify a `2 minutes` time window starting from `08:00:00`.

In [15]:
simulation_datetime = datetime.fromisoformat('2023-08-24T08:00:00')
(latitude, longitude) = (50.746708, 7.074405)
(seconds, meters) = (120, 1000)
car_positions = query(client, simulation_datetime, vehicle_type, latitude, longitude, seconds, meters)
car_positions_fset = FeatureSet.from_geojson(car_positions)
car_positions_sdf = car_positions_fset.sdf
car_positions_sdf

,trip,person,trip_time,OBJECTID,SHAPE
0,1562,966,2023-08-24T08:00:00,1,"{""x"": 7.08169, ""y"": 50.75054, ""spatialReferenc..."
1,2813,1738,2023-08-24T08:00:00,2,"{""x"": 7.06715, ""y"": 50.74203, ""spatialReferenc..."
2,6476,4048,2023-08-24T08:00:00,3,"{""x"": 7.07926, ""y"": 50.74946, ""spatialReferenc..."
3,6051,3781,2023-08-24T08:00:00,4,"{""x"": 7.08263, ""y"": 50.75098, ""spatialReferenc..."
4,1365,845,2023-08-24T08:00:00,5,"{""x"": 7.07359, ""y"": 50.74461, ""spatialReferenc..."
...,...,...,...,...,...
1832,5751,3598,2023-08-24T08:01:59,1833,"{""x"": 7.06613, ""y"": 50.74851, ""spatialReferenc..."
1833,1405,871,2023-08-24T08:01:59,1834,"{""x"": 7.07217, ""y"": 50.73929, ""spatialReferenc..."
1834,7133,4455,2023-08-24T08:01:59,1835,"{""x"": 7.07291, ""y"": 50.74686, ""spatialReferenc..."
1835,4651,2914,2023-08-24T08:01:59,1836,"{""x"": 7.07389, ""y"": 50.74639, ""spatialReferenc..."


## Map the car positions nearby

In [19]:
bonn_map = gis.map('Bonn, Germay')
car_positions_sdf.spatial.plot(bonn_map, ms=3)
bonn_map

MapView(layout=Layout(height='400px', width='100%'))